The `pyrit.memory` module provides functionality to keep track of the conversation history. In a nutshell, this can be used as follows

In [1]:

from pyrit.memory import DuckDBMemory
from pyrit.models import ChatMessage

memory = DuckDBMemory()
message_list = [
    ChatMessage(role="user", content="Hi, chat bot! This is my initial prompt."),
    ChatMessage(role="assistant", content="Nice to meet you! This is my response."),
]
next_message = ChatMessage(role="user", content="Wonderful! This is my second prompt to the chat bot.")
message_list.append(next_message)
memory.add_chat_messages_to_memory(conversations=message_list, conversation_id="11111")


# To retrieve the items from memory

memory.get_chat_messages_with_conversation_id(conversation_id="11111")

[ChatMessage(role='user', content='this is updtaed field'),
 ChatMessage(role='assistant', content='this is updtaed field'),
 ChatMessage(role='user', content='this is updtaed field'),
 ChatMessage(role='user', content='Hi, chat bot! This is my initial prompt.'),
 ChatMessage(role='assistant', content='Nice to meet you! This is my response.'),
 ChatMessage(role='user', content='Wonderful! This is my second prompt to the chat bot.')]

In [2]:
# update by conversation_id
update_fileds = {"content": "this is updtaed field"}
memory.update_entries_by_conversation_id(conversation_id="11111", update_fields=update_fileds)

True

In [3]:
# To retrieve the items from memory
memory.get_chat_messages_with_conversation_id(conversation_id="11111")

[ChatMessage(role='user', content='this is updtaed field'),
 ChatMessage(role='assistant', content='this is updtaed field'),
 ChatMessage(role='user', content='this is updtaed field'),
 ChatMessage(role='user', content='this is updtaed field'),
 ChatMessage(role='assistant', content='this is updtaed field'),
 ChatMessage(role='user', content='this is updtaed field')]

In [4]:
# Cleanup memory resources
memory.dispose_engine()